## Problem Statement

The City of Johannesburg  is perceived as  a crime ridden city in South Africa that most student outside of the city are reluctant  to come and study in Johannesburg.The aim of this project is explore neighborhoods  in the city of Johannesburg where student can stay safely and enjoy the rest of their  student life in the city.

This analysis  will assist any student who has never been to Johannesburg South Africa to apply to local universities with ease ,knowing that there are safe neighborhoods to stay in and venues  for student entertainment in the city.




## Data Description

To support my analysis i will use Foursquare API and machine learning algorithm to explore venues and  neighborhoods in Johannesburg by:
    
- Finding the safest neighborhood  based on crime statistics 
- Geographic locations showing Johannesburg city  coordinates 
- Choosing popular stundent accomodation in the neigborhood

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # librabry to handle requests
from geopy.geocoders import Nominatim 


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
 
from bs4 import BeautifulSoup

import folium


print('Libraries imported.')

Libraries imported.


# Lets get data for South African Provinces and Capital cities

In [3]:
# The code was removed by Watson Studio for sharing.

,Province,Capital,Coordinates
0,Eastern Cape,Bhisho,32°51′12″S 27°26′10″E﻿ / ﻿32.85333°S 27.43611°E
1,Free State,Bloemfontein,29°06′58″S 26°12′51″E﻿ / ﻿29.11611°S 26.21417°E
2,Gauteng,Johannesburg,26°12′22″S 28°02′24″E﻿ / ﻿26.20611°S 28.04000°E
3,KwaZulu-Natal,Pietermaritzburg,29°36′05″S 30°22′56″E﻿ / ﻿29.60139°S 30.38222°E
4,Limpopo,Polokwane,23°54′45″S 29°27′21″E﻿ / ﻿23.91250°S 29.45583°E
5,Mpumalanga,Nelspruit,25°26′15″S 30°58′19″E﻿ / ﻿25.43750°S 30.97194°E
6,North West,Mahikeng,25°50′03″S 25°36′42″E﻿ / ﻿25.83417°S 25.61167°E
7,Northern Cape,Kimberley,28°44′10″S 24°46′08″E﻿ / ﻿28.73611°S 24.76889°E
8,Western Cape,Cape Town,33°55′27″S 18°25′07″E﻿ / ﻿33.92417°S 18.41861°E


# Exploring Provinces by crime 

In [4]:
List_url = 'https://businesstech.co.za/news/lifestyle/340529/the-30-worst-areas-for-crime-in-south-africa'
source = requests.get(List_url).text

soup = BeautifulSoup(source, 'xml')
table=soup.find('table')
column_names=['Station','Province','Reported crimes 2019','Change from 2018',]
df = pd.DataFrame(columns=column_names)


In [5]:
df.head()

,Station,Province,Reported crimes 2019,Change from 2018


In [6]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==4:
        df.loc[len(df)] = row_data
df.head(100)


,Station,Province,Reported crimes 2019,Change from 2018
0,Cape Town Central,Western Cape,13 747,-10.9%
1,JHB Central,Gauteng,12 199,-0.2%
2,Mitchells Plain,Western Cape,11 374,-0.1%
3,Park Road,Free State,10 664,+11.0%
4,Honeydew,Gauteng,9 981,-5.0%
5,Durban Central,KZN,9 798,-3.4%
6,Pretoria Central,Gauteng,8 941,+1.6%
7,Witbank,Mpumalanga,8 916,-1.3%
8,Hillbrow,Gauteng,8 252,-4.7%
9,Sunnyside,Gauteng,7 833,+4.8%


# Selecting Gauteng Data only

In [7]:
Gauteng_data = df[df['Province'] =='Gauteng']
Gauteng_data.head(10)

,Station,Province,Reported crimes 2019,Change from 2018
1,JHB Central,Gauteng,12 199,-0.2%
4,Honeydew,Gauteng,9 981,-5.0%
6,Pretoria Central,Gauteng,8 941,+1.6%
8,Hillbrow,Gauteng,8 252,-4.7%
9,Sunnyside,Gauteng,7 833,+4.8%
12,Sandton,Gauteng,7 181,-5.7%
15,Dobsonville,Gauteng,7 009,+4.5%
19,Midrand,Gauteng,6 905,-6.9%
20,Kempton Park,Gauteng,6 900,+1.2%
22,Brooklyn,Gauteng,6 608,-3.3%


### **Exploring Neighborhoods using Foursquare data**

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# Selecting Midrand as a student location

In [5]:
address = 'Midrand, South Africa'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-25.999262 28.125912


In [2]:
! pip install folium

     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011


# Showing Map of Midrand and its latitude and Longitude values

In [28]:
map_Midrand = folium.Map(location=[latitude, longitude], zoom_start=11)
label = '{}, {}'.format('Johannesburg','Midrand')
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [-25.999262, 28.125912],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Midrand)  
map_Midrand
    

# Map of Midrand University

In [11]:

address = 'Midrand University'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


-26.019534999999998 28.1337591


In [12]:
# create a new dataframe for Midrand University
University_data = df[df['Station'] =='Johannesburg']


In [29]:
# create map of Midrand Univeristy  using latitude and longitude values

map_University = folium.Map(location=[latitude, longitude], zoom_start=11)
label = '{}, {}'.format('Johannesburg','University')
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [-26.019534999999998, 28.1337591],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_University)  
map_University

map_University

# Explore Nearby Venues in Midrand

In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['suburb', 
                   'Latitude', 
                   'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [70]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'
url

'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'

In [71]:
results = requests.get(url).json()
results

{'meta': {'code': 400,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '5f554f24539a1348c5f15435'},
 'response': {}}